# Convolutional Layers

> Compassion is a verb.
>
> – Thich Nhat Hanh

Today we will have a first look convolutional layers. Convolutional Neural Networks were first introduced in the 1980s by Yann LeCun and have been at the heart of most image related AI development ever since. 

While the latest research might change this

[An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://openreview.net/forum?id=YicbFdNTTy) 

I am sure the power of convolutions will remain a powerful techinque for any AI image wizards for quite some time.

A nice concise introduction of convolutional layers can be found on this excellent pyimagesearch blog by *Adrian Rosebrock*:

https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

For a more thorough explanation we refer to Chapter 14 of the Tensorflow bible:

[Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1492032646/&tag=jusdhaquo-20)

Here we - as usual - just play around with this technique a bit ...

